In [20]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import random
import os
import shutil

In [17]:
root = '/Volumes/TriveStorage/code/trive-image-recognition/autohub_latest'
# autohub_ids.txt was generated by autohub.js
# text file for domestic cars
# ids = [line.replace('R', '').strip() for line in open('autohub_ids.txt')]
# text file for foreign cars
ids = [line.replace('R', '').strip() for line in open('autohub_ids_foreign.txt')]
targets = [f'http://www.autohub.co.kr/buy/detailView.asp?DemoNo={x}' for x in ids]
random.shuffle(targets)

for target in targets:
    result = requests.get(target)
    if result.status_code == 200:
        try:
            soup = BeautifulSoup(result.content)
            title = soup.find_all('div', class_='info-header')[0].find_all('strong')[0].get_text()
            image_container = soup.find('ul', {'id': 'ulImgList'})
            images = list(set([x['src'].replace('small', 'large') for x in image_container.find_all('img') if '/CarPhoto/' in x['src']]))
            dst = os.path.join(root, title)
            if not os.path.exists(dst):
                os.mkdir(dst)
            for url in images:
                try:
                    response = requests.get(url, stream=True)
                    if not response.ok:
                        continue
                    with open(os.path.join(dst, url.split('CarPhoto/large')[1].replace('/', '')), 'wb') as handle:
                        for block in response.iter_content(1024):
                            if not block:
                                break
                            handle.write(block)
                except Exception as e:
                    print('}}}', e)
        except Exception as e:
            print('>>>', e)
    
    
    
    

>>> [Errno 2] No such file or directory: '/Volumes/TriveStorage/code/trive-image-recognition/autohub_latest/벤츠 SEL/SEC560 SEL'
}}} HTTPConnectionPool(host='www.autohub.co.kr', port=80): Max retries exceeded with url: /AttEdit/CarPhoto/large/8B07/8B07000217R10.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x113f693c8>: Failed to establish a new connection: [Errno 60] Operation timed out',))
}}} HTTPConnectionPool(host='www.autohub.co.kr', port=80): Max retries exceeded with url: /AttEdit/CarPhoto/large/8C07/8C07001844R12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x111a16208>: Failed to establish a new connection: [Errno 60] Operation timed out',))
}}} HTTPConnectionPool(host='www.autohub.co.kr', port=80): Max retries exceeded with url: /AttEdit/CarPhoto/large/9107/9107000978R18.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x111920cc0>: Failed to establish a new connection: [Errno 6

In [23]:
with open('autohub_names.txt', 'w') as f:
    for folder in [x for x in os.listdir(root) if x != '.DS_Store']:
        path = os.path.join(root, folder)
        if len(os.listdir(path)) < 2:
            shutil.rmtree(path)
        else:
            f.write(f'{folder}\n')